In [66]:
"""
@misc{home-credit-credit-risk-model-stability,
    author = {Daniel Herman, Tomas Jelinek, Walter Reade, Maggie Demkin, Addison Howard},
    title = {Home Credit - Credit Risk Model Stability},
    publisher = {Kaggle},
    year = {2024},
    url = {https://kaggle.com/competitions/home-credit-credit-risk-model-stability}
}
"""

'\n@misc{home-credit-credit-risk-model-stability,\n    author = {Daniel Herman, Tomas Jelinek, Walter Reade, Maggie Demkin, Addison Howard},\n    title = {Home Credit - Credit Risk Model Stability},\n    publisher = {Kaggle},\n    year = {2024},\n    url = {https://kaggle.com/competitions/home-credit-credit-risk-model-stability}\n}\n'

In [67]:
"""
Home Credit - Credit Risk Model Stability
Create a model measured against feature stability over time
The goal of this competition is to predict which clients are more likely to default on their loans. 
The evaluation will favor solutions that are stable over time.
"""

'\nHome Credit - Credit Risk Model Stability\nCreate a model measured against feature stability over time\nThe goal of this competition is to predict which clients are more likely to default on their loans. \nThe evaluation will favor solutions that are stable over time.\n'

In [68]:
"""
Lack of credit history could indicate youth or a preference for using cash.
Traditional data scarcity often leads to loan application rejections.
Data science is essential in determining loan repayment capabilities, potentially making loans more accessible to those in need.
Consumer finance providers utilize various statistical and machine learning methods in scorecards to assess loan risk.
Scorecards must be updated regularly due to the constant change in client behavior, making the process time-consuming.
The stability of a scorecard is crucial; a performance decline could mean issuing loans to higher-risk clients.
Early detection of loan repayment issues is challenging and usually visible only after loan due dates.
Balancing the stability and performance of scorecards is necessary before their implementation.
Home Credit, established in 1997, aims to offer responsible lending to people without significant credit history.
Home Credit's efforts in financial inclusion have previously been showcased in a Kaggle competition.
Improving scorecard assessments could help consumer finance providers accept more loan applications, enhancing financial access for historically underserved groups.
"""

"\nLack of credit history could indicate youth or a preference for using cash.\nTraditional data scarcity often leads to loan application rejections.\nData science is essential in determining loan repayment capabilities, potentially making loans more accessible to those in need.\nConsumer finance providers utilize various statistical and machine learning methods in scorecards to assess loan risk.\nScorecards must be updated regularly due to the constant change in client behavior, making the process time-consuming.\nThe stability of a scorecard is crucial; a performance decline could mean issuing loans to higher-risk clients.\nEarly detection of loan repayment issues is challenging and usually visible only after loan due dates.\nBalancing the stability and performance of scorecards is necessary before their implementation.\nHome Credit, established in 1997, aims to offer responsible lending to people without significant credit history.\nHome Credit's efforts in financial inclusion have 

In [69]:
"""
For each case_id in the test set, you must predict a probability for the target score. The file should contain a header and have the following format:

case_id,score
57543,0.1
57544,0.9
57545,0.5
etc.


Submission file must be named submission.csv

"""

'\nFor each case_id in the test set, you must predict a probability for the target score. The file should contain a header and have the following format:\n\ncase_id,score\n57543,0.1\n57544,0.9\n57545,0.5\netc.\n\n\nSubmission file must be named submission.csv\n\n'

In [70]:
"""
 predicting default of clients based on internal and external information that are available for each client. Scoring is 
 performed using custom metric that not only evaluates the AUC of predictions but also considers the stability of predictions 
 model across the data range of the test set.
"""

'\n predicting default of clients based on internal and external information that are available for each client. Scoring is \n performed using custom metric that not only evaluates the AUC of predictions but also considers the stability of predictions \n model across the data range of the test set.\n'

In [71]:
import polars as pl #Libreria para manipulacion de datos, fast wrote in rust https://docs.pola.rs 
import numpy as np #Libreria para manipulacion de datos 
import pandas as pd #Libreria para manipulacion de datos 
import lightgbm as lgb #Libreria para modelos de machine learning,  is a gradient boosting framework that uses tree based learning algorithms. It is designed to be distributed and efficient
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score 



In [72]:
#Se cambia la ruta de acuerdo a la ubicacion de los datos
dataPath = "C:/Users/diego/Documents/UP/Negocios/DataProyecto/"

Cargar datos

In [73]:
def establecer_tipos_de_datos_tabla(df: pl.DataFrame) -> pl.DataFrame:
    # implementar aquí todos los tipos de datos deseados para las tablas
    # lo siguiente es solo un ejemplo
    for columna in df.columns:
        # la última letra del nombre de la columna te ayudará a determinar el tipo
        if columna[-1] in ("P", "A"):
            df = df.with_columns(pl.col(columna).cast(pl.Float64).alias(columna)) #Pasa las columnas a float64

    return df

In [74]:
def convertir_cadenas(df: pd.DataFrame) -> pd.DataFrame:
    for columna in df.columns:  
        if df[columna].dtype.name in ['object', 'string']:
            df[columna] = df[columna].astype("string").astype('category')
            categorias_actuales = df[columna].cat.categories
            nuevas_categorias = categorias_actuales.to_list() + ["Desconocido"]
            nuevo_tipo = pd.CategoricalDtype(categories=nuevas_categorias, ordered=True)
            df[columna] = df[columna].astype(nuevo_tipo)
            
    return df

In [75]:
ruta_base = dataPath + "csv_files/train/"

# Leer el archivo base de entrenamiento
tabla_base_entrenamiento = pl.read_csv(ruta_base + "train_base.csv")

# Leer y concatenar los archivos estáticos de entrenamiento
archivo_estatico_1 = pl.read_csv(ruta_base + "train_static_0_0.csv").pipe(establecer_tipos_de_datos_tabla)
archivo_estatico_2 = pl.read_csv(ruta_base + "train_static_0_1.csv").pipe(establecer_tipos_de_datos_tabla)
train_static = pl.concat([archivo_estatico_1, archivo_estatico_2], how="vertical_relaxed")

# Leer otros archivos de entrenamiento
entrenamiento_estatico_cb = pl.read_csv(ruta_base + "train_static_cb_0.csv").pipe(establecer_tipos_de_datos_tabla)
train_person_1 = pl.read_csv(ruta_base + "train_person_1.csv").pipe(establecer_tipos_de_datos_tabla)
train_credit_bureau_b_2 = pl.read_csv(ruta_base + "train_credit_bureau_b_2.csv").pipe(establecer_tipos_de_datos_tabla)

In [76]:
ruta_base_prueba = dataPath + "csv_files/test/"

# Leer el archivo base de prueba
train_basetable = pl.read_csv(ruta_base_prueba + "test_base.csv")

# Leer y concatenar los archivos estáticos de prueba
archivo_estatico_prueba_1 = pl.read_csv(ruta_base_prueba + "test_static_0_0.csv").pipe(establecer_tipos_de_datos_tabla)
archivo_estatico_prueba_2 = pl.read_csv(ruta_base_prueba + "test_static_0_1.csv").pipe(establecer_tipos_de_datos_tabla)
archivo_estatico_prueba_3 = pl.read_csv(ruta_base_prueba + "test_static_0_2.csv").pipe(establecer_tipos_de_datos_tabla)
prueba_estatica = pl.concat([archivo_estatico_prueba_1, archivo_estatico_prueba_2, archivo_estatico_prueba_3], how="vertical_relaxed")

# Leer otros archivos de prueba
train_static_cb = pl.read_csv(ruta_base_prueba + "test_static_cb_0.csv").pipe(establecer_tipos_de_datos_tabla)
test_person_1 = pl.read_csv(ruta_base_prueba + "test_person_1.csv").pipe(establecer_tipos_de_datos_tabla)
test_credit_bureau_b_2 = pl.read_csv(ruta_base_prueba + "test_credit_bureau_b_2.csv").pipe(establecer_tipos_de_datos_tabla)

Feature engineering

In [77]:
# Necesitamos usar funciones de agregación en tablas con profundidad > 1, es decir, tablas que contienen la columna num_group1 o
# también la columna num_group2.
caracteristicas_persona_1 = train_person_1.group_by("case_id").agg(
    # Obtenemos el valor máximo de la columna 'mainoccupationinc_384A' para cada 'case_id'
    pl.col("mainoccupationinc_384A").max().alias("maximo_mainoccupationinc_384A"),
    # Verificamos si alguna vez 'incometype_1044T' es 'SELFEMPLOYED' para cada 'case_id'
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("alguna_vez_selfemployed")
)

# Aquí num_group1=0 tiene un significado especial, es la persona que solicitó el préstamo.
caracteristicas_persona_2 = train_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    # Filtramos las filas donde 'num_group1' es 0
    pl.col("num_group1") == 0
# Eliminamos la columna 'num_group1' y renombramos la columna 'housetype_905L' a 'tipo_casa_persona'
).drop("num_group1").rename({"housetype_905L": "tipo_casa_persona"})

# Aquí tenemos num_goup1 y num_group2, por lo que necesitamos agregar de nuevo.
caracteristicas_credito_b_2 = train_credit_bureau_b_2.group_by("case_id").agg(
    # Obtenemos el valor máximo de la columna 'pmts_pmtsoverdue_635A' para cada 'case_id'
    pl.col("pmts_pmtsoverdue_635A").max().alias("maximo_pmts_pmtsoverdue_635A"),
    # Verificamos si alguna vez 'pmts_dpdvalue_108P' es mayor que 31 para cada 'case_id'
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("alguna_vez_pmts_dpdvalue_108P_mayor_31")
)

# Procesaremos en este ejemplo solo las columnas de tipo A y M, por lo que necesitamos seleccionarlas.
columnas_estaticas_seleccionadas = [col for col in train_static.columns if col[-1] in ("A", "M")]
print(columnas_estaticas_seleccionadas)

columnas_estaticas_cb_seleccionadas = [col for col in train_static_cb.columns if col[-1] in ("A", "M")]
print(columnas_estaticas_cb_seleccionadas)

# Unimos todas las tablas.
data = train_basetable.join(
    train_static.select(["case_id"]+columnas_estaticas_seleccionadas), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+columnas_estaticas_cb_seleccionadas), how="left", on="case_id"
).join(
    caracteristicas_persona_1, how="left", on="case_id"
).join(
    caracteristicas_persona_2, how="left", on="case_id"
).join(
    caracteristicas_credito_b_2, how="left", on="case_id"
)

['amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'avginstallast24m_3658937A', 'avglnamtstart24m_4525187A', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'credamount_770A', 'currdebt_22A', 'currdebtcredtyperange_828A', 'disbursedcredamount_1113A', 'downpmt_116A', 'inittransactionamount_650A', 'lastapprcommoditycat_1041M', 'lastapprcommoditytypec_5251766M', 'lastapprcredamount_781A', 'lastcancelreason_561M', 'lastotherinc_902A', 'lastotherlnsexpense_631A', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectcredamount_222A', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'maininc_215A', 'maxannuity_159A', 'maxannuity_4075009A', 'maxdebt4_972A', 'maxinstallast24m_3658928A', 'maxlnamtstart6m_4525199A', 'maxoutstandbalancel12m_4187113A', 'maxpmtlast3m_4525190A', 'previouscontdistrict_112M', 'price_1097A', 'sumoutstandtotal_3546847A', 'sumoutstandtotalest_4493215A', 'totaldebt_9A', 'totalsettled_863A', 'totinstallas

In [78]:
# Necesitamos usar funciones de agregación en tablas con profundidad > 1, es decir, tablas que contienen la columna num_group1 o
# también la columna num_group2.
test_person_1_feats_1 = test_person_1.group_by("case_id").agg(
    # Obtenemos el valor máximo de la columna 'mainoccupationinc_384A' para cada 'case_id'
    pl.col("mainoccupationinc_384A").max().alias("maximo_mainoccupationinc_384A"),
    # Verificamos si alguna vez 'incometype_1044T' es 'SELFEMPLOYED' para cada 'case_id'
    (pl.col("incometype_1044T") == "SELFEMPLOYED").max().alias("alguna_vez_selfemployed")
)

# Aquí num_group1=0 tiene un significado especial, es la persona que solicitó el préstamo.
test_person_1_feats_2 = test_person_1.select(["case_id", "num_group1", "housetype_905L"]).filter(
    # Filtramos las filas donde 'num_group1' es 0
    pl.col("num_group1") == 0
# Eliminamos la columna 'num_group1' y renombramos la columna 'housetype_905L' a 'tipo_casa_persona'
).drop("num_group1").rename({"housetype_905L": "tipo_casa_persona"})

# Aquí tenemos num_goup1 y num_group2, por lo que necesitamos agregar de nuevo.
test_credit_bureau_b_2_feats = test_credit_bureau_b_2.group_by("case_id").agg(
    # Obtenemos el valor máximo de la columna 'pmts_pmtsoverdue_635A' para cada 'case_id'
    pl.col("pmts_pmtsoverdue_635A").max().alias("maximo_pmts_pmtsoverdue_635A"),
    # Verificamos si alguna vez 'pmts_dpdvalue_108P' es mayor que 31 para cada 'case_id'
    (pl.col("pmts_dpdvalue_108P") > 31).max().alias("alguna_vez_pmts_dpdvalue_108P_mayor_31")
)

# Unimos todas las tablas.
data_submission = train_basetable.join(
    prueba_estatica.select(["case_id"]+columnas_estaticas_seleccionadas), how="left", on="case_id"
).join(
    train_static_cb.select(["case_id"]+columnas_estaticas_cb_seleccionadas), how="left", on="case_id"
).join(
    test_person_1_feats_1, how="left", on="case_id"
).join(
    test_person_1_feats_2, how="left", on="case_id"
).join(
    test_credit_bureau_b_2_feats, how="left", on="case_id"
)

In [81]:
# Primero, obtenemos los 'case_id' únicos y los mezclamos.
case_ids = data["case_id"].unique().shuffle(seed=1)

# Luego, dividimos los 'case_id' en conjuntos de entrenamiento y prueba.
case_ids_train, case_ids_test = train_test_split(case_ids, train_size=0.6, random_state=1)
# Dividimos el conjunto de prueba en conjuntos de validación y prueba.
case_ids_valid, case_ids_test = train_test_split(case_ids_test, train_size=0.5, random_state=1)

# Creamos una lista vacía para almacenar las columnas de predicción.
cols_pred = []
# Recorremos todas las columnas en los datos.
for col in data.columns:
    # Si el último carácter de la columna es una letra mayúscula y el resto de la columna es minúscula,
    # añadimos la columna a la lista de columnas de predicción.
    if col[-1].isupper() and col[:-1].islower():
        cols_pred.append(col)

# Imprimimos las columnas de predicción.
print(cols_pred)

# Definimos una función para convertir los datos de Polars a Pandas.
def from_polars_to_pandas(case_ids: pl.DataFrame) -> pl.DataFrame:
    # Filtramos los datos por 'case_id' y seleccionamos las columnas 'case_id', 'WEEK_NUM' y 'target'.
    # Convertimos los datos a Pandas.
    return (
        data.filter(pl.col("case_id").is_in(case_ids))[["case_id", "WEEK_NUM", "target"]].to_pandas(),
        # Filtramos los datos por 'case_id' y seleccionamos las columnas de predicción.
        # Convertimos los datos a Pandas.
        data.filter(pl.col("case_id").is_in(case_ids))[cols_pred].to_pandas(),
        # Filtramos los datos por 'case_id' y seleccionamos la columna 'target'.
        # Convertimos los datos a Pandas.
        data.filter(pl.col("case_id").is_in(case_ids))["target"].to_pandas()
    )

# Usamos la función definida anteriormente para convertir los conjuntos de entrenamiento, validación y prueba a Pandas.
base_train, X_train, y_train = from_polars_to_pandas(case_ids_train)
base_valid, X_valid, y_valid = from_polars_to_pandas(case_ids_valid)
base_test, X_test, y_test = from_polars_to_pandas(case_ids_test)

# Convertimos las cadenas en los conjuntos de entrenamiento, validación y prueba.
for df in [X_train, X_valid, X_test]:
    df = convertir_cadenas(df)

TypeError: cannot use `__getitem__` on Series of dtype Int64 with argument (array([0, 3, 1, 7, 8, 5]), Ellipsis) of type 'tuple'

In [ ]:
# Leemos los archivos CSV y los cargamos en DataFrames de Polars.
train_basetable = pl.read_csv(dataPath + "csv_files/train/train_base.csv")
# Concatenamos los archivos CSV 'train_static_0_0.csv' y 'train_static_0_1.csv' verticalmente.
train_static = pl.concat(
    [
        pl.read_csv(dataPath + "csv_files/train/train_static_0_0.csv").pipe(establecer_tipos_de_datos_tabla),
        pl.read_csv(dataPath + "csv_files/train/train_static_0_1.csv").pipe(establecer_tipos_de_datos_tabla),
    ],
    how="vertical_relaxed",
)
train_static_cb = pl.read_csv(dataPath + "csv_files/train/train_static_cb_0.csv").pipe(establecer_tipos_de_datos_tabla)
train_person_1 = pl.read_csv(dataPath + "csv_files/train/train_person_1.csv").pipe(establecer_tipos_de_datos_tabla)
train_credit_bureau_b_2 = pl.read_csv(dataPath + "csv_files/train/train_credit_bureau_b_2.csv").pipe(establecer_tipos_de_datos_tabla)

In [ ]:
print("Train")
X_train.shape

In [ ]:
print("Valid")
X_valid.shape

In [ ]:
print("Test")
X_test.shape

Entrenando Modelo

In [ ]:
#Se puede hacer con LightGBM pero, podemos hacerlo por random forest u otras opciones

Para subir el archivo al concurso